<a href="https://colab.research.google.com/github/khushee-g/EEG/blob/main/With_Shiny_Apps_Band_Pass_Exponentiated_Muse_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # Linear algebra
import pandas as pd
# ModelLing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt
import pandas as pd

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

fs = 256
lowcut = 0.5
highcut = 4

file_path = file_info["datapath"]
data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

num_nan_raw_AF8 = data['RAW_AF8'].isnull().sum()
num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()
num_nan_raw_TP9 = data['RAW_TP9'].isnull().sum()
num_nan_raw_TP10 = data['RAW_TP10'].isnull().sum()

if num_nan_raw_AF8 > 0:
    data['RAW_AF8'] = data['RAW_AF8'].interpolate(method='linear')

if num_nan_raw_AF7 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')

if num_nan_raw_TP9 > 0:
    data['RAW_TP9'] = data['RAW_TP9'].interpolate(method='linear')

if num_nan_raw_TP10 > 0:
    data['RAW_TP10'] = data['RAW_TP10'].interpolate(method='linear')


filtered_delta_AF8 = bandpass_filter(data['RAW_AF8'].values, lowcut, highcut, fs, order=5)
filtered_delta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP9 = bandpass_filter(data['RAW_TP9'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP10 = bandpass_filter(data['RAW_TP10'].values, lowcut, highcut, fs, order=5)

print(filtered_delta_TP10)


[ 2.00767501e+73  1.80000531e+73  1.59439966e+73 ... -1.46678676e+35
 -1.26937680e+35 -1.09212825e+35]


<ipython-input-2-3dd559401d72>:21: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
fs = 256
lowcut = 4
highcut = 8

num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()

if num_nan_raw_TP10 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')


filtered_theta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)


In [ ]:
#Delta AF7-AF8 coherence - corresponds to COH.A.delta.a.FP1.b.FP2

from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = filtered_delta_AF8
channel_2 = filtered_delta_AF7

#channel_1_mean = np.nanmean(channel_1)
#channel_2_mean = np.nanmean(channel_2)

#channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
#channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF7DeltaAF8': Cxy})

coherence_AF8_AF7 = coherence_data['Coherence DeltaAF7DeltaAF8'].mean()
coherence_AF8_AF7 *=100
print(coherence_AF8_AF7)


29.682911147695158


<ipython-input-4-ce0fe9c778df>:5: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
#Delta AF8-TP9 coherence - corresponds to COH.A.delta.f.F4.h.T3



from scipy import signal

data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')

channel_1 = filtered_delta_AF8
channel_1 = filtered_delta_TP9

channel_1_mean = np.nanmean(channel_1)
channel_2_mean = np.nanmean(channel_2)

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF8DeltaTP9': Cxy})

coherence_AF8_TP9 = coherence_data['Coherence DeltaAF8DeltaTP9'].mean()
coherence_AF8_TP9 *=100
print(coherence_AF8_TP9)

20.436728137103636


<ipython-input-5-5dca52ce387b>:7: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mindMonitor_2024-11-15--21-52-00.csv')


In [ ]:
risk = 0
#coeff intercept + coeff channel val * (coherence*100)
#e^x


#COH.A.delta.a.FP1.b.FP2
AF7_AF8 = 0.0246*(coherence_AF8_AF7) - 1.7527
AF7_AF8 = np.exp(AF7_AF8)
AF7_AF8 -=1
risk += AF7_AF8



#COH.A.delta.f.F4.h.T3
AF8_TP9 = -0.0040*(coherence_AF8_TP9) + 0.1324
AF8_TP9 = np.exp(AF8_TP9)
AF8_TP9 -=1
risk += AF8_TP9



#AB.C.alpha.q.T6
TP10 = 0.0097*filtered_delta_TP10.mean() - 0.1531
TP10 = np.exp(TP10)
TP10 -=1
risk += TP10



#AB.B.theta.c.F7
AF7 = -0.0152 *filtered_theta_AF7.mean() + 0.2759
AF7 = np.exp(AF7)
AF7 -=1
risk += AF7


print(risk)

if risk > 0:
  prediction_val = (risk-1)*100
  print("You are " + str(prediction_val) + "% more likely to have behavioral disorder")

else:
  prediction_val = (risk+1)*-100
  print("You are " + str(prediction_val) + " % less likely to have behavioral disorder")




-1.2705729414576912
You are 27.057294145769117 % less likely to have behavioral disorder


In [ ]:
!pip install marginaleffects

import numpy as np
import polars as pl
from marginaleffects import *
import statsmodels.formula.api as smf
mtcars = pl.read_csv("/content/mindMonitor_2024-11-15--21-52-00.csv")
mod = smf.ols("mpg ~ hp", data = mtcars).fit()

print(mod.summary().as_text())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of matplotlib to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 100.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotli

In [ ]:
import mimetypes
from math import ceil
from typing import List

from shiny.express import input, render, ui

MAX_SIZE = 50000
ui.input_file("file1", "Choose a file to upload:", multiple=True)
ui.input_radio_buttons("type", "Type:", ["Text", "Binary"])


def format_hexdump(data: bytes) -> str:
    hex_vals = ["{:02x}".format(b) for b in data]
    hex_vals = group_into_blocks(hex_vals, 16)
    hex_vals = [" ".join(row) for row in hex_vals]
    hex_vals = "\n".join(hex_vals)
    return hex_vals


def group_into_blocks(x: List[str], blocksize: int):
    """
    Given a list, return a list of lists, where the inner lists each have `blocksize`
    elements.
    """
    return [
        x[i * blocksize : (i + 1) * blocksize] for i in range(ceil(len(x) / blocksize))
    ]


@render.code
def file_content():
    file_infos = input.file1()
    if not file_infos:
        return

    # file_infos is a list of dicts; each dict represents one file. Example:
    # [
    #   {
    #     'name': 'data.csv',
    #     'size': 2601,
    #     'type': 'text/csv',
    #     'datapath': '/tmp/fileupload-1wnx_7c2/tmpga4x9mps/0.csv'
    #   }
    # ]
    out_str = ""
    for file_info in file_infos:
        out_str += (
            "=" * 47
            + "\n"
            + file_info["name"]
            + "\nMIME type: "
            + str(mimetypes.guess_type(file_info["name"])[0])
        )
        if file_info["size"] > MAX_SIZE:
            out_str += f"\nTruncating at {MAX_SIZE} bytes."

        out_str += "\n" + "=" * 47 + "\n"

        if input.type() == "Text":
            with open(file_info["datapath"], "r") as f:
                out_str += f.read(MAX_SIZE)
        else:
            with open(file_info["datapath"], "rb") as f:
                data = f.read(MAX_SIZE)
                out_str += format_hexdump(data)

    return out_str







from pathlib import Path

import pandas

from shiny import reactive
from shiny.express import render, ui


@reactive.calc
def dat():
    infile = input.file1()
    return pandas.read_csv(infile)


with ui.navset_card_underline():

    with ui.nav_panel("Data frame"):
        @render.data_frame
        def frame():
            # Give dat() to render.DataGrid to customize the grid
            return dat()

    with ui.nav_panel("Table"):
        @render.table
        def table():
            return dat()
















import numpy as np # Linear algebra
import pandas as pd
# ModelLing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics


import numpy as np
from scipy.signal import butter, filtfilt
import pandas as pd

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

fs = 256
lowcut = 0.5
highcut = 4

infile = input.file1()
data = pandas.read_csv(infile)

num_nan_raw_AF8 = data['RAW_AF8'].isnull().sum()
num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()
num_nan_raw_TP9 = data['RAW_TP9'].isnull().sum()
num_nan_raw_TP10 = data['RAW_TP10'].isnull().sum()

if num_nan_raw_AF8 > 0:
    data['RAW_AF8'] = data['RAW_AF8'].interpolate(method='linear')

if num_nan_raw_AF7 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')

if num_nan_raw_TP9 > 0:
    data['RAW_TP9'] = data['RAW_TP9'].interpolate(method='linear')

if num_nan_raw_TP10 > 0:
    data['RAW_TP10'] = data['RAW_TP10'].interpolate(method='linear')


filtered_delta_AF8 = bandpass_filter(data['RAW_AF8'].values, lowcut, highcut, fs, order=5)
filtered_delta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP9 = bandpass_filter(data['RAW_TP9'].values, lowcut, highcut, fs, order=5)
filtered_delta_TP10 = bandpass_filter(data['RAW_TP10'].values, lowcut, highcut, fs, order=5)

print(filtered_delta_TP10)


fs = 256
lowcut = 4
highcut = 8

num_nan_raw_AF7 = data['RAW_AF7'].isnull().sum()

if num_nan_raw_TP10 > 0:
    data['RAW_AF7'] = data['RAW_AF7'].interpolate(method='linear')


filtered_theta_AF7 = bandpass_filter(data['RAW_AF7'].values, lowcut, highcut, fs, order=5)

#Delta AF7-AF8 coherence - corresponds to COH.A.delta.a.FP1.b.FP2

from scipy import signal

data = pandas.read_csv(infile)

channel_1 = filtered_delta_AF8
channel_2 = filtered_delta_AF7

#channel_1_mean = np.nanmean(channel_1)
#channel_2_mean = np.nanmean(channel_2)

#channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
#channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF7DeltaAF8': Cxy})

coherence_AF8_AF7 = coherence_data['Coherence DeltaAF7DeltaAF8'].mean()
coherence_AF8_AF7 *=100
print(coherence_AF8_AF7)


#Delta AF8-TP9 coherence - corresponds to COH.A.delta.f.F4.h.T3



from scipy import signal

data = pandas.read_csv(infile)

channel_1 = filtered_delta_AF8
channel_1 = filtered_delta_TP9

channel_1_mean = np.nanmean(channel_1)
channel_2_mean = np.nanmean(channel_2)

channel_1 = np.nan_to_num(channel_1, nan=np.nanmean(channel_1))
channel_2 = np.nan_to_num(channel_2, nan=np.nanmean(channel_2))

fs = 256
f, Cxy = signal.coherence(channel_1, channel_2, fs=fs)  # fs is sampling frequency

coherence_data = pd.DataFrame({'Frequency': f, 'Coherence DeltaAF8DeltaTP9': Cxy})

coherence_AF8_TP9 = coherence_data['Coherence DeltaAF8DeltaTP9'].mean()
coherence_AF8_TP9 *=100
print(coherence_AF8_TP9)



risk = 0
#coeff intercept + coeff channel val * (coherence*100)
#e^x


#COH.A.delta.a.FP1.b.FP2
AF7_AF8 = 0.0246*(coherence_AF8_AF7) - 1.7527
AF7_AF8 = np.exp(AF7_AF8)
AF7_AF8 -=1
risk += AF7_AF8



#COH.A.delta.f.F4.h.T3
AF8_TP9 = -0.0040*(coherence_AF8_TP9) + 0.1324
AF8_TP9 = np.exp(AF8_TP9)
AF8_TP9 -=1
risk += AF8_TP9



#AB.C.alpha.q.T6
TP10 = 0.0097*filtered_delta_TP10.mean() - 0.1531
TP10 = np.exp(TP10)
TP10 -=1
risk += TP10



#AB.B.theta.c.F7
AF7 = -0.0152 *filtered_theta_AF7.mean() + 0.2759
AF7 = np.exp(AF7)
AF7 -=1
risk += AF7


print(risk)

if risk > 0:
  prediction_val = (risk-1)*100
  print("You are " + str(prediction_val) + "% more likely to have behavioral disorder")

else:
  prediction_val = (risk+1)*-100
  print("You are " + str(prediction_val) + " % less likely to have behavioral disorder")




